<a href="https://colab.research.google.com/github/yutaro-tanaka-yt2705/ag-cgan/blob/main/3_build_own_cgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import our common libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import gzip
import sys
import warnings

In [6]:
#Import our DL libraries
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import regularizers, layers
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Activation, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential, load_model, save_model, model_from_json
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from torch.optim import Adam
import torch.cuda as cuda
import torch.nn as nn
import torch
from torch.distributions import Normal

In [17]:
#Import our data
data = pd.read_csv('/content/drive/MyDrive/artificial_genome_project/test_dataset.csv')
data.head()

,ID,pair,X_1343510_CA_C,X_2464340_G_A,X_844565_T_G,X_1443587_C_T,X_846946_A_G,X_155783945_C_T,X_2348441_C_T,X_1495791_C_G,...,X_1494541_A_G,X_1494970_G_A,X_2272724_A_C,X_2441570_T_C,X_1420810_T_C,X_719380_A_G,X_1115457_A_G,X_2449045_T_C,X_1165297_A_G,population_code
0,HG00096,A,0,0,0,1,0,1,1,1,...,0,0,1,0,0,0,0,0,1,10.0
1,HG00096,B,0,0,1,1,1,1,0,1,...,0,0,1,0,0,0,1,0,0,10.0
2,HG00097,A,0,0,1,1,1,1,0,1,...,0,0,0,0,1,0,0,0,0,10.0
3,HG00097,B,0,0,1,1,0,1,0,0,...,0,0,1,0,0,0,0,1,0,10.0
4,HG00099,A,0,0,1,1,1,1,0,0,...,0,0,1,1,1,0,0,1,0,10.0


In [ ]:
#Split and load data
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [13]:
dataloader = torch.utils.data.DataLoader(data, batch_size=hp.batchsize, num_workers=1,
                                         shuffle=True, drop_last=True, pin_memory=True)

In [10]:
#Implement hyperparameters
class Hyperparameter:
    num_classes: int        = 10
    batchsize: int          = 128
    num_epochs: int         = 20
    latent_size: int        = 32
    n_critic: int           = 5
    critic_size: int        = 1024
    generator_size: int     = 1024
    critic_hidden_size: int = 1024
    gp_lambda: float        = 10.
        
hp = Hyperparameter()

In [12]:
#Implement Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.latent_embedding = nn.Sequential(
            nn.Linear(hp.latent_size, hp.generator_size // 2),
        )
        self.condition_embedding = nn.Sequential(
            nn.Linear(hp.num_classes, hp.generator_size // 2),
        )
        self.tcnn = nn.Sequential(
        nn.ConvTranspose2d( hp.generator_size, hp.generator_size, 4, 1, 0),
        nn.BatchNorm2d(hp.generator_size),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d( hp.generator_size, hp.generator_size // 2, 3, 2, 1),
        nn.BatchNorm2d(hp.generator_size // 2),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d( hp.generator_size // 2, hp.generator_size // 4, 4, 2, 1),
        nn.BatchNorm2d(hp.generator_size // 4),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d( hp.generator_size // 4, 1, 4, 2, 1),
        nn.Tanh()
        )   
    def forward(self, latent, condition):
        vec_latent = self.latent_embedding(latent)
        vec_class = self.condition_embedding(condition)
        combined = torch.cat([vec_latent, vec_class], dim=1).reshape(-1, hp.generator_size, 1, 1)
        return self.tcnn(combined)

In [11]:
#Implement Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.condition_embedding = nn.Sequential(
            nn.Linear(hp.num_classes, hp.Discriminator_size * 4),
        )
        self.cnn_net = nn.Sequential(
        nn.Conv2d(1, hp.Discriminator_size // 4, 3, 2),
        nn.InstanceNorm2d(hp.Discriminator_size // 4, affine=True),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Conv2d(hp.Discriminator_size // 4, hp.Discriminator_size // 2, 3, 2),
        nn.InstanceNorm2d(hp.Discriminator_size // 2, affine=True),
        nn.LeakyReLU(0.2, inplace=True),   
        nn.Conv2d(hp.Discriminator_size // 2, hp.Discriminator_size, 3, 2),
        nn.InstanceNorm2d(hp.Discriminator_size, affine=True),
        nn.LeakyReLU(0.2, inplace=True),
        nn.Flatten(),
        )
        self.Discriminator_net = nn.Sequential(
        nn.Linear(hp.Discriminator_size * 8, hp.Discriminator_hidden_size),
        nn.LeakyReLU(0.2, inplace=True),   
        nn.Linear(hp.Discriminator_hidden_size, 1),
        )
        
    def forward(self, image, condition):
        vec_condition = self.condition_embedding(condition)
        cnn_features = self.cnn_net(image)
        combined = torch.cat([cnn_features, vec_condition], dim=1)
        return self.Discriminator_net(combined)

TypeError: ignored